# **Modelos para detección de transacciones financieras fraudulentas**

## **Descripción**

En este notebook se entrenan y evalúan varios modelos de Machine Learning (Logistic Regression, Random Forest y XGBoost) para la detección de transacciones fraudulentas en tarjetas de crédito. El objetivo es comparar distintas técnicas de clasificación y escoger la mejor en términos de recall, precision y otras métricas adecuadas para un problema altamente desbalanceado.  

Los apartados tratados son:
- Carga del dataset tras un EDA previo.
- División en entrenamiento y test (estratificado).
- Entrenamiento de varios algoritmos.
- Evaluación con métricas (matriz de confusión, AUPRC, etc.).
- Comparación de los resultados.
- Conclusiones finales.

## **Módulos**

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    precision_recall_curve, auc, roc_auc_score
)

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

## **Cargo dataset tras EDA**

In [77]:
df = pd.read_csv("creditcard_clean.csv") 
print("Tamaño del dataset:", df.shape)

Tamaño del dataset: (283726, 31)


## **Preparación de features y target**

In [78]:
X = df.drop(columns=['class'])  # El resto de columnas son predictoras
y = df['class']                 # Clase (0: No Fraude, 1: Fraude)

## **División en entrenamiento y test (estratificado por estar desbalanceado)**

In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=7
)

print("\nDistribución de clases en entrenamiento:")
print(y_train.value_counts(normalize=True))
print("\nDistribución de clases en test:")
print(y_test.value_counts(normalize=True))


Distribución de clases en entrenamiento:
class
0    0.998332
1    0.001668
Name: proportion, dtype: float64

Distribución de clases en test:
class
0    0.998336
1    0.001664
Name: proportion, dtype: float64


## **Función para evaluar los modelos**

In [80]:
def evaluar_modelo(model, X_test, y_test, umbral=0.5):
    """
    Imprime matriz de confusión, metrics, AUPRC, ROC-AUC, etc.
    umbral es el corte para predecir 0 o 1 a partir de la probabilidad
    """
    
    y_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades de la clase positiva (fraude=1)
    y_pred = (y_proba >= umbral).astype(int)     # Predicciones binarias, según umbral
    
    cm = confusion_matrix(y_test, y_pred)        # Matriz de confusión
    
    # Precision, Recall, AUPRC
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    pr_auc = auc(recall, precision)
    
    roc_auc = roc_auc_score(y_test, y_proba)     # ROC AUC
    
    # Clasification Report
    clf_report = classification_report(y_test, y_pred, digits=4)
    
    print("Matriz de Confusión:")
    print(cm)
    print("\nReporte de Clasificación:")
    print(clf_report)
    print(f"AUPRC = {pr_auc:.4f}")
    print(f"ROC-AUC = {roc_auc:.4f}")

## **Entrenamiento de los modelos**

### **Regresión logística**

In [81]:
print("============================================")
print("     MODELO 1: LOGISTIC REGRESSION")
print("============================================")
lr = LogisticRegression(
    solver='lbfgs',
    max_iter=10000,
    random_state=7
)

lr.fit(X_train, y_train)
evaluar_modelo(lr, X_test, y_test, umbral=0.3)

     MODELO 1: LOGISTIC REGRESSION
Matriz de Confusión:
[[70801    13]
 [   35    83]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0     0.9995    0.9998    0.9997     70814
           1     0.8646    0.7034    0.7757       118

    accuracy                         0.9993     70932
   macro avg     0.9320    0.8516    0.8877     70932
weighted avg     0.9993    0.9993    0.9993     70932

AUPRC = 0.7358
ROC-AUC = 0.9825


### **Random Forest**

In [82]:
from sklearn.model_selection import GridSearchCV
print("\n============================================")
print("     MODELO 2: RANDOM FOREST")
print("============================================")

param_grid = {
    'n_estimators': [25, 50, 100],
    'max_depth': [None],
    'max_features': ['sqrt', 'log2']
}

rf_model = RandomForestClassifier(random_state=7)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='average_precision',  # métrica para datos desbalanceados
    cv=3,                         # 3-fold cross-validation
)

grid_search.fit(X_train, y_train)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación (average_precision):", grid_search.best_score_)

# Entreno un RandomForest con los mejores parámetros
best_rf = grid_search.best_estimator_

evaluar_modelo(best_rf, X_test, y_test, umbral=0.3)


     MODELO 2: RANDOM FOREST
Mejores parámetros: {'max_depth': None, 'max_features': 'log2', 'n_estimators': 100}
Mejor puntuación (average_precision): 0.8243824897511809
Matriz de Confusión:
[[70802    12]
 [   23    95]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0     0.9997    0.9998    0.9998     70814
           1     0.8879    0.8051    0.8444       118

    accuracy                         0.9995     70932
   macro avg     0.9438    0.9025    0.9221     70932
weighted avg     0.9995    0.9995    0.9995     70932

AUPRC = 0.8443
ROC-AUC = 0.9432


### **XGBoost (XGBClassifier)**

In [83]:
from sklearn.model_selection import RandomizedSearchCV

print("\n============================================")
print("     MODELO 3: XGBOOST CLASSIFIER")
print("============================================")

param_dist = {
    'n_estimators': [25, 50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_clf = XGBClassifier(random_state=7, eval_metric='logloss')

random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    scoring='average_precision',
    n_iter=5,      # número de combinaciones que testeará
    cv=3,          # cross-validation
    verbose=1,
    random_state=7,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Mejores parámetros (XGB):", random_search.best_params_)
print("Mejor puntuación (avg_precision):", random_search.best_score_)

best_xgb = random_search.best_estimator_
evaluar_modelo(best_xgb, X_test, y_test, umbral=0.3)



     MODELO 3: XGBOOST CLASSIFIER
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Mejores parámetros (XGB): {'subsample': 1.0, 'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.2, 'colsample_bytree': 1.0}
Mejor puntuación (avg_precision): 0.8398824278001081
Matriz de Confusión:
[[70801    13]
 [   25    93]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0     0.9996    0.9998    0.9997     70814
           1     0.8774    0.7881    0.8304       118

    accuracy                         0.9995     70932
   macro avg     0.9385    0.8940    0.9150     70932
weighted avg     0.9994    0.9995    0.9994     70932

AUPRC = 0.8250
ROC-AUC = 0.9854


## **Comparación de resultados**

In [84]:
models = [('LogisticReg', lr), ('RandomForest', best_rf), ('XGBoost', best_xgb)]
metricas = []

for name, model in models:
    y_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    pr_auc = auc(recall, precision)
    roc_auc = roc_auc_score(y_test, y_proba)
    metricas.append((name, pr_auc, roc_auc))

print("\n============================================")
print("     COMPARACIÓN DE MODELOS (AUPRC/ROC-AUC)")
print("============================================")
for m in metricas:
    print(f"{m[0]} -> AUPRC: {m[1]:.4f} | ROC-AUC: {m[2]:.4f}")


     COMPARACIÓN DE MODELOS (AUPRC/ROC-AUC)
LogisticReg -> AUPRC: 0.7358 | ROC-AUC: 0.9825
RandomForest -> AUPRC: 0.8443 | ROC-AUC: 0.9432
XGBoost -> AUPRC: 0.8250 | ROC-AUC: 0.9854


## **Conclusiones finales**

- **Regresión logística**  
  De 118 fraudes en test, detecta 83 (recall=70.3%), dejando 35 sin detectar (falsos negativos).  
  Cada vez que predice fraude, acierta un 86.5% (precision=86.5%).  
  El AUPRC de ~0.74 indica un desempeño decente en la curva de Precisión-Recall, y su ROC-AUC ~0.98 aunque en problemas desbalanceados como este, el AUPRC y el recall de la clase minoritaria son más relevantes. 

- **Random Forest**  
  Detecta 95 de los 118 fraudes (recall=80.5%), dejando 23 sin detectar.  
  La precisión es ~88.8%, es decir, de los que clasifica como fraude, casi 9 de cada 10 son realmente fraudes.  
  El AUPRC ~0.84 es muy bueno en un problema tan desbalanceado. Random Forest maneja bien la clase minoritaria.  
  Su ROC-AUC es 0.94 (ligeramente menor que en Regresión y XGBoost), pero en fraude a menudo importa más la AUPRC y la recall de la clase 1.  

- **XGBoost**  
  Detecta 93 de los 118 fraudes (recall=78.8%), dejando 25 sin detectar.  
  La precisión es ~87.7%, ligeramente menor que Random Forest.  
  El AUPRC ~0.83 y el ROC-AUC ~0.98 también son muy buenos.  
  Se queda un poco por debajo de Random Forest en recall, pero tiene una ROC-AUC más alta.  

| Modelo          | Recall (%) | Precisión (%) | AUPRC  | ROC-AUC |
|----------------|-----------|--------------|--------|--------|
| Random Forest  | **80.5**  | **88.8**     | **0.8443** | 0.9432  |
| XGBoost        | 78.8      | 87.7         | 0.8250 | **0.9854** |
| Logistic       | 70.3      | 86.5         | 0.7358 | 0.9825  |

**Mayor Recall -> Clasifico más fraudes como fraudes**   
**Mayor Precisión -> Menos falsos positivos (Detecta como fraudes transacciones que eran buenas)**   
**AUPRC (Área bajo la curva de Precisión-Recall) -> Muy útil en datasets desbalanceados**    
**ROC-AUC (Área bajo la curva ROC)**   

En términos de negocio, Random Forest deja menos fraudes sin detectar (23 falsos negativos), con muy pocos falsos positivos (12). XGBoost deja 25 FN, Logistic 35 FN. Por lo tanto, considerando que la prioridad es detectar el mayor número de fraudes (alto recall) con un buen balance de precisión, el modelo **Random Forest** destaca como la mejor opción entre estos tres, seguido de XGBoost en segundo lugar.